
# A study on docking in a recently produced hA2B receptor crystal structure using machine learning
Benthe Smit (s2619806). Team mates: Tamara Jordens (s2190745), Nienke Vreman (s2333937).

This report consists of 3 parts: Bioinformatics, Cheminformatics & Machine Learning and Docking. Our target of interest is the human A2B receptor.

In [1]:
import nglview
import os
import shutil
from scripts import viewer

from Bio.PDB import PDBList, MMCIFParser, Select, PDBIO
import Bio.Align
import os

from pathlib import Path
from warnings import filterwarnings
import time

import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect

from scripts import viewer
from scripts import bio_align

import pandas as pd
import numpy as np
import glob
import sys
from sklearn import svm, metrics, clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import auc, accuracy_score, recall_score
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt


filterwarnings("ignore")
# Fix seed for reproducible results
SEED = 22
#seed_everything(SEED)

#from pymol import cmd
import py3Dmol

from vina import Vina

#from meeko import MoleculePreparation
#from meeko import obutils

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

#import prolif
#from prolif.plotting.network import LigNetwork

import sys, os, shutil
sys.path.insert(1, '/project/jhllei001/JHL_data/Jupyter_Dock/utilities')
# Note I had to comment out pymol, openmm and pdbfixer, hope it doesn't break later
#from utils import fix_protein, getbox, generate_ledock_file, pdbqt_to_sdf, dok_to_sdf

import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False

/opt/conda/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/Bio/Data/SCOPData.py:18: BiopythonDeprecationWarning: The 'Bio.Data.SCOPData' module will be deprecated in a future release of Biopython in favor of 'Bio.Data.PDBData.
  warnings.warn(



# 1. Bioinformatics

### 1.1.1 Introduction

Adenosine is a biochemical molecule that is distributed all over the body, in various pathways. In the central nervous system (CNS) is promotes sleep by modulating neuroactivity. Adenosine is able to regulate the cardiovascular system as well, by inducing vasodilation, which leads to an increasing blood flow and tissue oxygenation. Moreover, as adenosine activates immune cells, it decreases the production of cytokines. Therefore, it reduces inflammation and regulates immune response.[1]

Both adenosine and adenosine triphosphate (ATP) are the endogenous ligands to the adenosine receptors (ARs). This class of receptors is divided into four main subtypes: A1, A2A, A2B, and A3 ARs. All four of the ARs belong to the G-protein coupled receptors (GPCRs) and consist of a transmembrane protein containing 7 parts, of which the A1R and the A3R and the A2AR and the A2BR are the most similar. The A1R and A3R activate an inhibitory G protein and the A2AR and the A2BR activate stimulatory proteins. Though many ligands have been discovered for the ARs, only a handful are selective for one of the receptors. A crystal structure is available for all the ARs, except for the A3R.[2]

In this study we will focus on targeting the A2BR. The A2BR has a low affinity for adenosine and was therefore considered less important in the past.[3] Hence, the A2BR is interesting target. There are a limited amount of clinically approved drugs for the AR are available at the moment, most of which are antagonists.[4] Currently, there are 4 structures available on the protein database (PDB), of which 8HDO has the highest resolution. 8HDO is co-crystallized with a synthetic agonist,[5] while 8HDP is bound to adenosine.[6] Both structures were released in january 2023. In may 2023, two more structures were released. 

This study will mainly focus on finding a partial agonist to the target. As agonists will control inflammation in the brain and many other tissues. One of the most well-known partial agonists is BAY 60–6583. This  synthetic ligand has been co-crystalized with the A2BR as well in 8HDO. It will therefore be used as a basis for further improvement. On PDB the following information has been extracted on the primary structure: The A2BR has a sequence length of 332 amino acids and a mass of 36.333 kDa. The specific PDB structure (8HDO/P29275) is UniProtKB reviewed and has evidence at protein level.


### 1.1.2 Related proteins (off-targets), based on sequence

Firstly, related proteins were investigated to predict possible off-target effects. Using Basic Local Alignment Search Tool (BLAST) on UniProt, the following protein was found:

With the identity of 99.7%, the Western lowland gorilla has the most similar protein (G3QH04). It length is 332 amino acids and it has a mass of 36.351 kDa. However the protein is not UniProtKB reviewed and it is inferred from homology.
    
In the human body, the most similar protein is the A2AR (P29274) as the identity is 59.4%. It has a length of 412 amino acids and a mass of 44.707 kDa. This structure is UniProtKB reviewed and has evidence at protein level.

The most similar target to the human A2BR (hA2BR) is the A2BR in the Western lowland gorilla. This was expected as humans are from similar descent as gorilla's and it is the same protein. However, during this study, the human A2AR (hA2A) will be used to check for off-target effects, as that will provide more relevant information.
    
Using the 'Accession' numbers, the proteins were aligned based on their: similar residues, hydrophobic residues, negative residues, positive residues and aromatic residues.

![image info](img/Similarity_hA2B_A2B_A2A_1.jpg)
*Figure 1.1: Alignment by UniProt based on similarity. The structures are presented in the following order: hA2AR, hA2BR, A2BR in gorilla.*

![image info](img/Similarity_hA2B_A2B_A2A_2.jpg)
*Figure 1.2: Alignment by UniProt based on hydrophobic residues. The structures are presented in the following order: hA2AR, hA2BR, A2BR in gorilla.*

![image info](img/Similarity_hA2B_A2B_A2A_3.jpg)
*Figure 1.3: Alignment by UniProt based on negative residues. The structures are presented in the following order: hA2AR, hA2BR, A2BR in gorilla.*

![image info](img/Similarity_hA2B_A2B_A2A_4.jpg)
*Figure 1.4: Alignment by UniProt based on positive residues. The structures are presented in the following order: hA2AR, hA2BR, A2BR in gorilla.*

![image info](img/Similarity_hA2B_A2B_A2A_5.jpg)
*Figure 1.5: Alignment by UniProt based on aromatic residues. The structures are presented in the following order: hA2AR, hA2BR, A2BR in gorilla.*

The hA2BR and the A2BR in the gorilla are the most similar again.


### 1.1.3 Related proteins (off-targets), based on structure

Since a crystal structure of the target as recently been released, a 3D similarity search is now possible. The proteins will be compared based on their tertiary structure (3D structure) rather than a sequence similarity.

On http://www.rcsb.org/pdb/home/home.do, the A2BR was found using the PDB identifier. The most similar accession was 8HDP. The two protein complexes were aligned using https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html.

![image info](img/Realinged_8HDO_8HDP.jpg)
*Figure 1.6: Alignment of 8HDO and 8HDP. The realigned RMSD was 0.6186 Å.*

This is a very low RMSD value, which means the structures are very similar. This is logical, as it is the same protein, just different crystal structures.


## 1.2 Retrieving a 3D structure

The protein was prepared by downloading the structure from PDB. 

Firstly, a working directory was made in the home directory:

In [2]:
HOMEDIR = str(Path.home())
os.chdir(HOMEDIR)
# We need to check whether the directory is there
try:
    os.mkdir('Bioinformatics')
except:
    print("Directory already exists")
os.chdir('Bioinformatics')

Directory already exists


The PDB code of the hA2BR was specified and the 3D structure was generated.

In [3]:
TARGET_PDB_ID = "8HDO"
view = nglview.show_pdbid(TARGET_PDB_ID)
view.center()
view

NGLWidget()

It  is nice to have the overview of the structure, but since we are interested in designing new drugs, it makes more sense to have a closer look at the co-crystallized ligand. For this structure that is BAY 60–6583. The three letter amino acid code from the RCSB is I5D.

In [4]:
LIGAND_CODE = "I5D"

view.center(LIGAND_CODE)
view

NGLWidget(n_components=1)

Next, the residues that are within 5 Angstrom of the ligand are shown.

In [5]:
viewer.show_residues_around(view, selection=LIGAND_CODE)
view

NGLWidget(n_components=1)

The viewer shows that there are multiple types of interactions present between the protein and the ligand, such as pi-stacking, hydrophobic interactions and hydrogen bonds. Later, the Protein-Ligand Interaction Profiler will be used to identify the specific interactions.

There are no hydrogens present in the structures as these are normally not resolved in the crystal structure.

In the next stage, the hydrogens were added. The protein and ligand were split and saved seperately. For this, biopython was used. https://biopython.org/docs/1.75/api/Bio.html

Firstly, the coordinates were downloaded from RCSB.

In [6]:
pdbl = PDBList()
pdbl.retrieve_pdb_file(TARGET_PDB_ID, pdir=TARGET_PDB_ID)

Structure exists: '8HDO/8hdo.cif' 


'8HDO/8hdo.cif'

A BioPython object was generated from the coordinates.

In [7]:
parser = MMCIFParser()
structure = parser.get_structure("TARGETPROT",'{}/{}.cif'.format(TARGET_PDB_ID,TARGET_PDB_ID.lower()))

The ligand is saved.

In [8]:
class ResSelect(Select):
    def accept_residue(self, residue):
        if residue.get_resname() == LIGAND_CODE:
            return 1
        else:
            return 0

class NonHetSelect(Select):
    def accept_residue(self, residue):
        return 1 if residue.id[0] == " " else 0

io = PDBIO()
io.set_structure(structure)
io.save("ligand-{}.pdb".format(LIGAND_CODE), ResSelect())
io.save("protein-{}.pdb".format(TARGET_PDB_ID), NonHetSelect())

LePro, which is part of the LeDock program (https://en.wikipedia.org/wiki/LeDock), is used to add hydrogens to the structure.

The protein was prepared:

In [9]:
command = '../CBR_teaching/bin/lepro protein-{}.pdb'.format(TARGET_PDB_ID)
os.system(command)
shutil.move('pro.pdb','{}_prepped.pdb'.format(TARGET_PDB_ID))

'8HDO_prepped.pdb'

In [10]:
# combine protein and ligand files
filenames = [
'{}_prepped.pdb'.format(TARGET_PDB_ID),
"ligand-{}.pdb".format(LIGAND_CODE)
]
with open('{}-complex.pdb'.format(TARGET_PDB_ID), 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                if not "END" in line:
                    outfile.write(line)

In [11]:
with open('{}-complex.pdb'.format(TARGET_PDB_ID)) as f:
    view = nglview.show_file(f, ext="pdb")
    
view.center(LIGAND_CODE)
viewer.show_residues_around(view, selection=LIGAND_CODE)
view

NGLWidget()

Here, the protein is generated again.

Now, the procedure is repeated for the most similar target that you identified (the highest scoring hit from the PDB):

In [12]:
OFF_TARGET_PDB_ID = "5IU4"
OFF_TARGET_LIGAND = "ZMA"  

pdbl = PDBList()
pdbl.retrieve_pdb_file(OFF_TARGET_PDB_ID, pdir=OFF_TARGET_PDB_ID)

parser = MMCIFParser()
structure = parser.get_structure("TARGETPROT",'{}/{}.cif'.format(OFF_TARGET_PDB_ID,OFF_TARGET_PDB_ID.lower()))

class ResSelect(Select):
    def accept_residue(self, residue):
        if residue.get_resname() == OFF_TARGET_LIGAND:
            return 1
        else:
            return 0

io = PDBIO()
io.set_structure(structure)
io.save("ligand-{}.pdb".format(OFF_TARGET_LIGAND), ResSelect())
io.save("protein-{}.pdb".format(OFF_TARGET_PDB_ID), NonHetSelect())

Structure exists: '5IU4/5iu4.cif' 


In [13]:
command = '../CBR_teaching/bin/lepro protein-{}.pdb'.format(OFF_TARGET_PDB_ID)
os.system(command)
shutil.move('pro.pdb','{}_prepped.pdb'.format(OFF_TARGET_PDB_ID))

'5IU4_prepped.pdb'

In [14]:
# combine protein and ligand files
filenames = [
'{}_prepped.pdb'.format(OFF_TARGET_PDB_ID),
"ligand-{}.pdb".format(OFF_TARGET_LIGAND)
]
with open('{}-complex.pdb'.format(OFF_TARGET_PDB_ID), 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                if not "END" in line:
                    outfile.write(line)

The protein was prepared.

In [15]:
with open('{}-complex.pdb'.format(OFF_TARGET_PDB_ID)) as f:
    view = nglview.show_file(f, ext="pdb")
    
view.center(OFF_TARGET_LIGAND)
viewer.show_residues_around(view, selection=OFF_TARGET_LIGAND)
view

NGLWidget()

## 1.3 Alignment
The hA2BR and the hA2AR were aligned by first generating the alignment object. The alignment was conducted to predict possible off-target effects of future novel ligands.

In [16]:
from Bio import pairwise2
from Bio.Seq import Seq 
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

# Get the structures
PDBCODE_1 = '8HDO' # Name of the first structure
PDBCODE_2 = '5IU4' # Name of the second structure

import requests
data = requests.get(f'https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{PDBCODE_1}').json()[PDBCODE_1.lower()]
SEQ1 = (data[0]['sequence'])
SEQ1 = Seq(SEQ1)

data = requests.get(f'https://www.ebi.ac.uk/pdbe/api/pdb/entry/molecules/{PDBCODE_2}').json()[PDBCODE_2.lower()]
SEQ2 = (data[0]['sequence'])
SEQ2 = Seq(SEQ2)

alignments = pairwise2.align.globalxx(SEQ1, SEQ2)

for align1, align2, score, begin, end in alignments:
    filename = "alignment.fasta"
    with open(filename, "w") as handle:
        handle.write(">SEQ1\n%s\n>SEQ2\n%s\n" % (align1, align2))

print(alignments[0])

Alignment(seqA='------------MGC--------L--------GN----------SKTED--QRNEEK---------AQREANKM--IEKQLQKDKQVYR---A-----T---------HRL--L------L-LGADN-SGKSTIVKQM---------R-I-------YHVNSG---------I--------FE----TKF---------QVD-KVNFHMFDV-G-A--QRD---ERRKWI-Q--C-FN-DVTAIIFV-----VDSSDY-NR-----L-------------------QE-A-LN-DFKSI-W---N-NRWLRTIS-VILFLN-K-----Q--DLLAE--KV-----L-AG-KS----KI-EDYF--P---E---FARYTTPE---D--------ATP-----EPGEDPR-V-----------T-R-AKYFIRDEF---L-RI--ST------A--SGD-----GRHY----CY--P-H----FT--C---S-----------VD---TEN--------AR-RI--FNDCRD-----IIQRM-H-LRQY-ELL----------------', seqB='DYKDDDDGAPPIMG-SSVYITVELAIAVLAILGNVLVCWAVWLNS----NLQ-N---VTNYFVVSLA---A---ADI---L-----V--GVLAIPFAITISTGFCAACH--GCLFIACFVLVL-A--QS--S-I----FSLLAIAIDRYIAIAIPLRY--N-GLVTGTRAAGIIAICWVLSF-AIGLT--PMLGWNNCGQ--PK--------EGKAHSQ--GCGE-----GQVACLF-EDV-----VPMNYMV----YFN-FFACVLVPLLLMLGVYLRIFAAARRQ-LADL-ED----NWETLNDN--L----KVI----EKADNAAQVKD--A-LTK-MRAAALDA-QK-ATPPK-LED--KSPDSPEMKDF-R-----HGFDILVGQIDDA--LKLANE-G----

The proteins were visually aligned using https://www.ncbi.nlm.nih.gov/Structure/icn3d/full.html. The RMSD was 4.085 Å. This is quite high. The structures are thus not very similar.

![image info](img/Realinged_8HDO_5IU4.jpg)
*Figure 1.7: The alignment of the hA2BR and the hA2AR.*

## Interactions between the hA2BR and BAY 60-6583

The Protein-Ligand Interaction Profiler (PLIP, https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index) was used to identify the interactions between the protein and the ligand.

![image info](img/8HDO_I5D_R_408.png)
*Figure 1.8: The interactions between the hA2BR and BAY 60-6583 according to PLIP. The protein is represented in blue and the ligand in orange. The interactions are presents as follows: blue for hydrogen bonds, grey for hydrophobic interactions, green for pi-stacking.*

Finally, the interaction viewer on rcsb itself was used to visualize the protein-ligand complex.

![image info](img/RCSB_8HDO.png)
*Figure 1.9: The hA2BR and BAY 60-6583 complex according to RCSB. Both the protein and ligand are presented in green.*

It seems that the pi-stacking between the ligand and residue PHE173 is very important to the binding. Just like the hydrophobic interactions between the ligand and LEU172, PHE173 and ILE276. Lastly, there are hydrogen bonds between THR89, THR89, ASN186, ASN254. 

PLIP states that there could be a hydrophobic interaction between the ligand and residue SER68. However, a serine residue is not known to produce such interactions. It is mainly used by proteins to establish hydrogenbond interactions, because of its free -OH group.


### References:

[1] Korkutata, M., & Lazarus, M. (2023). Adenosine A2A receptors and sleep (Vol. 170). International Review of Neurobiology. doi: 10.1016/bs.irn.2023.04.007.

[2] Vincenzi F, Pasquini S, Contri C, Cappello M, Nigro M, Travagli A, Merighi S, Gessi S, Borea PA, Varani K. Pharmacology of Adenosine Receptors: Recent Advancements. Biomolecules. 2023 Sep 14;13(9):1387. doi: 10.3390/biom13091387.

[3] Catarzi D, Varano F, Varani K, Vincenzi F, Pasquini S, Dal Ben D, Volpini R, Colotta V. Amino-3,5-Dicyanopyridines Targeting the Adenosine Receptors Ranging from Pan Ligands to Combined A1/A2B Partial Agonists. Pharmaceuticals (Basel). 2019 Oct 22;12(4):159. doi: 10.3390/ph12040159.

[4] Adenosine receptor A2B | DrugBank Online. (z.d.). DrugBank. Geraadpleegd op 8 november 2023, van https://go.drugbank.com/bio_entities/BE0000241.

[5] Bank, R. P. D. RCSB PDB - 8HDO: Structure of A2BR bound to Synthetic Agonists BAY 60-6583. Geraadpleegd op 8 november 2023, van https://www.rcsb.org/structure/8HDO.

[6] Bank, R. P. D. RCSB PDB - 8HDP: Structure of A2BR bound to Endogenous Agonists adenosine. Geraadpleegd op 8 november 2023, van https://www.rcsb.org/structure/8HDP.

[7] National Center for Biotechnology Information (2023). PubChem Compound Summary for CID 11717831, 2-({6-Amino-3,5-dicyano-4-[4-(cyclopropylmethoxy)phenyl]pyridin-2-yl}sulfanyl)acetamide. Retrieved November 8, 2023 from https://pubchem.ncbi.nlm.nih.gov/compound/bay-60-6583.

[8] Chen Y, Zhang J, Weng Y, Xu Y, Lu W, Liu W, Liu M, Hua T, Song G. Cryo-EM structure of the human adenosine A2B receptor-Gs signaling complex. Sci Adv. 2022 Dec 23;8(51):eadd3709. doi: 10.1126/sciadv.add3709.